In [ ]:
from pyspark.sql import SparkSession
import requests

In [ ]:
spark = SparkSession.builder \
    .appName("Read_Silver_Layer") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioLocalAccessKey") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioLocalSecretKey123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()
    
spark

In [ ]:
paths = {
    "customers": "s3a://etl-dag/silver/data/customers/",
    "transactions_static": "s3a://etl-dag/silver/data/transactions/static/",
    "transactions_aggregated": "s3a://etl-dag/silver/data/transactions/aggregated/",
    "products_static": "s3a://etl-dag/silver/data/products/static/",
    "products_aggregated": "s3a://etl-dag/silver/data/products/aggregated/"
}

dfs = {}
for name, path in paths.items():
    dfs[name] = spark.read.parquet(path)
    print(f"✅ Loaded: {name}")

In [ ]:
for name, df in dfs.items():
    print(f"\n📄 Preview: {name}")
    df.show(5, truncate=False)